# Update Read Todo State Test

## Complete Todo Management Workflow

This notebook demonstrates a complete todo management workflow using multiple states in sequence:

1. **READ_TODO**: Read existing todo content from file
2. **GENERATE**: Generate update instructions using LLM
3. **UPDATE_TODO**: Update the todo content based on instructions
4. **FINAL**: Complete the workflow

```lua
stateDiagram-v2
    INIT --> READ_TODO
    READ_TODO --> GENERATE
    GENERATE --> UPDATE_TODO
    UPDATE_TODO --> FINAL
```

```mermaid
stateDiagram-v2
    Direction LR
    INIT --> READ_TODO
    READ_TODO --> GENERATE
    GENERATE --> UPDATE_TODO
    UPDATE_TODO --> FINAL
```


In [1]:
from gai.asm import AgenticStateMachine
from gai.messages import Monologue
from gai.mcp.client import McpAggregatedClient
import tempfile
import os
from pathlib import Path

print("=== Complete Todo Management Workflow Test ===")

# Create initial todo content
initial_todo_content = """# My Project Todo List

## High Priority Tasks
- [ ] Fix authentication bug in login module
- [ ] Update API documentation for new endpoints
- [ ] Review and merge pull request #145

## Medium Priority Tasks
- [ ] Implement user profile editing feature
- [ ] Add comprehensive unit tests for payment system
- [ ] Optimize database query performance

## Low Priority Tasks
- [ ] Update project README with new setup instructions
- [ ] Clean up old log files and unused assets
- [ ] Organize project folder structure

## Completed This Week
- [x] Fixed CSS styling issues on mobile devices
- [x] Updated all npm dependencies to latest versions
- [x] Completed code review for authentication feature

---
*Last updated: 2025-08-03*
"""

# Create a temporary file with initial content
with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False) as f:
    f.write(initial_todo_content)
    workflow_todo_path = f.name

print(f"Created test todo file at: {workflow_todo_path}")
print(f"Initial content length: {len(initial_todo_content)} characters")

=== Complete Todo Management Workflow Test ===
Created test todo file at: /tmp/tmpghhpfz4w.md
Initial content length: 729 characters


In [2]:
# Define the GENERATE action function
async def generate_update_instructions(state):
    """Generate update instructions based on the current todo content."""
    from gai.llm.openai import AsyncOpenAI
    
    # Get the current todo content from state bag
    todo_content = state.machine.state_bag.get("todo_content", "")
    
    if not todo_content:
        # If no content, generate default instructions
        update_instruction = "Create a new todo list with high priority tasks for today"
        state.machine.state_bag["update_instruction"] = update_instruction
        return
    
    # Create LLM client
    llm_config = state.machine.state_bag["llm_config"]
    client = AsyncOpenAI(llm_config)
    
    # Create prompt for generating update instructions
    system_prompt = f"""
    You are a productivity assistant. Based on the current todo list below, generate specific update instructions.
    
    Current Todo List:
    {todo_content}
    
    Generate update instructions that would improve the todo list by:
    1. Marking some pending tasks as completed (be realistic)
    2. Adding 1-2 new urgent tasks
    3. Updating priorities based on urgency
    4. Adding estimated completion dates for key tasks
    
    Provide clear, specific instructions in a single paragraph.
    """
    
    try:
        response = await client.chat.completions.create(
            model=llm_config["model"],
            messages=[
                {"role": "system", "content": "You are a helpful productivity assistant."},
                {"role": "user", "content": system_prompt}
            ],
            max_tokens=200,
            temperature=0.7
        )
        
        update_instruction = response.choices[0].message.content.strip()
        state.machine.state_bag["update_instruction"] = update_instruction
        
        print(f"Generated update instruction: {update_instruction}")
        
    except Exception as e:
        print(f"Error generating instructions: {e}")
        # Fallback instruction
        update_instruction = "Mark the authentication bug as completed, add a new urgent task for security audit, and update due dates for all high priority items to this week."
        state.machine.state_bag["update_instruction"] = update_instruction

# Define the workflow with all 5 states
async def run_complete_workflow():
    """Run the complete todo management workflow."""
    
    monologue = Monologue()
    
    with AgenticStateMachine.StateMachineBuilder(
        """
        INIT --> READ_TODO
        READ_TODO --> GENERATE
        GENERATE --> UPDATE_TODO
        UPDATE_TODO --> FINAL
        """
    ) as builder:
        fsm = builder.build(
            {
                "INIT": {
                    "input_data": {
                        "todo_path": {"type": "getter", "dependency": "get_todo_path"},
                        "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                        "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                    }
                },
                "READ_TODO": {
                    "module_path": "gai.asm.states",
                    "class_name": "ReadToDoState",
                    "title": "READ_TODO",
                    "input_data": {
                        "todo_path": {"type": "state_bag", "dependency": "todo_path"},
                    },
                    "output_data": ["todo_content", "todo_file_path", "file_exists"],
                },
                "GENERATE": {
                    "module_path": "gai.asm.states",
                    "class_name": "PureActionState",
                    "title": "GENERATE",
                    "action": "generate_update_instructions",
                    "output_data": ["update_instruction"],
                },
                "UPDATE_TODO": {
                    "module_path": "gai.asm.states",
                    "class_name": "UpdateToDoState",
                    "title": "UPDATE_TODO",
                    "input_data": {
                        "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                        "todo_data": {"type": "state_bag", "dependency": "todo_content"},
                        "update_instruction": {"type": "state_bag", "dependency": "update_instruction"},
                        "todo_path": {"type": "state_bag", "dependency": "todo_path"},
                        "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
                    },
                    "output_data": ["streamer", "get_assistant_message", "updated_todo", "todo_file_path"],
                },
                "FINAL": {
                    "output_data": ["monologue", "updated_todo", "todo_content", "update_instruction"],
                },
            },
            get_todo_path=lambda state: workflow_todo_path,
            get_llm_config=lambda state: {
                "client_type": "anthropic",
                "model": "claude-sonnet-4-20250514",
                "max_tokens": 32000,
                "temperature": 0.7,
                "top_p": 0.95,
                "tools": True,
            },
            get_mcp_client=lambda state: McpAggregatedClient([]),
            generate_update_instructions=generate_update_instructions,
            monologue=monologue,
        )
    
    return fsm

# Run the workflow
fsm = await run_complete_workflow()

In [3]:
print("\n" + "="*60)
print("STEP 1: READ_TODO - Reading initial todo content")
print("="*60)

# Execute READ_TODO state
await fsm.run_async()

# Display read results
todo_content = fsm.state_bag.get("todo_content")
file_exists = fsm.state_bag.get("file_exists")
todo_file_path = fsm.state_bag.get("todo_file_path")

print(f"✅ READ_TODO completed")
print(f"📁 File path: {todo_file_path}")
print(f"📄 File exists: {file_exists}")
print(f"📝 Content length: {len(todo_content) if todo_content else 0} characters")

if todo_content:
    print("\n📋 Initial Todo Content:")
    print("-" * 50)
    print(todo_content[:500] + "..." if len(todo_content) > 500 else todo_content)
    print("-" * 50)


STEP 1: READ_TODO - Reading initial todo content


ValueError: State 'IS_TERMINATE' is not a registered state.

In [ ]:
print("\n" + "="*60)
print("STEP 2: GENERATE - Generating update instructions")
print("="*60)

# Reset state machine history before next execution
fsm.restart()

# Execute GENERATE state
await fsm.run_async()

# Display generation results
update_instruction = fsm.state_bag.get("update_instruction")

print(f"✅ GENERATE completed")
print(f"📝 Generated instruction length: {len(update_instruction) if update_instruction else 0} characters")

if update_instruction:
    print("\n🎯 Generated Update Instructions:")
    print("-" * 50)
    print(update_instruction)
    print("-" * 50)

In [ ]:
print("\n" + "="*60)
print("STEP 3: UPDATE_TODO - Updating todo content with generated instructions")
print("="*60)

# Reset state machine history before next execution
fsm.restart()

# Execute UPDATE_TODO state
await fsm.run_async()

# Stream the LLM response
print("🔄 Streaming LLM response:")
print("-" * 30)

async for chunk in fsm.state_bag["streamer"]:
    if hasattr(chunk, '__iter__') and not isinstance(chunk, str):
        for sub_chunk in chunk:
            if isinstance(sub_chunk, str):
                print(sub_chunk, end='', flush=True)
    elif isinstance(chunk, str):
        print(chunk, end='', flush=True)

print("\n" + "-" * 50)
print("✅ UPDATE_TODO completed")

# Display update results
updated_todo = fsm.state_bag.get("updated_todo")
if updated_todo:
    print(f"📝 Updated content available")
    updated_content = updated_todo.get("updated_content", "")
    print(f"📏 Updated content length: {len(updated_content)} characters")

In [ ]:
print("\n" + "="*60)
print("STEP 4: FINAL - Completing workflow and showing results")
print("="*60)

# Reset state machine history before next execution
fsm.restart()

# Execute FINAL state
await fsm.run_async()

print("✅ FINAL state completed")
print("🎉 Complete workflow finished successfully!")

# Display comprehensive results
print("\n" + "="*60)
print("WORKFLOW SUMMARY")
print("="*60)

# State execution summary
print("🔄 State Execution Path:")
for i, state_info in enumerate(fsm.state_history, 1):
    state_name = state_info.get("state", "Unknown")
    print(f"   {i}. {state_name}")

# Content comparison
initial_content = todo_content
final_updated_todo = fsm.state_bag.get("updated_todo")
final_content = final_updated_todo.get("updated_content", "") if final_updated_todo else ""

print(f"\n📊 Content Statistics:")
print(f"   📄 Initial content: {len(initial_content)} characters")
print(f"   📄 Final content: {len(final_content)} characters")
print(f"   📈 Change: {len(final_content) - len(initial_content):+d} characters")

# Instruction used
final_instruction = fsm.state_bag.get("update_instruction")
print(f"\n🎯 Update Instruction Used:")
print(f"   {final_instruction}")

# File information
final_file_path = fsm.state_bag.get("todo_file_path")
print(f"\n📁 File Information:")
print(f"   📂 Path: {final_file_path}")
print(f"   📝 File updated: {os.path.exists(final_file_path) if final_file_path else 'Unknown'}")

if final_content:
    print(f"\n📋 Final Updated Todo Content:")
    print("=" * 50)
    print(final_content)
    print("=" * 50)

---

## Test Case 2: Error Handling - Non-existent File

Test the workflow when starting with a non-existent todo file.


In [ ]:
print("=== Test Case 2: Non-existent File Workflow ===")

# Use a non-existent file path
non_existent_path = "/tmp/definitely_does_not_exist_workflow_test.md"

async def run_error_handling_workflow():
    """Test workflow with non-existent file."""
    
    monologue = Monologue()
    
    with AgenticStateMachine.StateMachineBuilder(
        """
        INIT --> READ_TODO
        READ_TODO --> GENERATE
        GENERATE --> UPDATE_TODO
        UPDATE_TODO --> FINAL
        """
    ) as builder:
        fsm = builder.build(
            {
                "INIT": {
                    "input_data": {
                        "todo_path": {"type": "getter", "dependency": "get_todo_path"},
                        "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                        "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                    }
                },
                "READ_TODO": {
                    "module_path": "gai.asm.states",
                    "class_name": "ReadToDoState",
                    "title": "READ_TODO",
                    "input_data": {
                        "todo_path": {"type": "state_bag", "dependency": "todo_path"},
                    },
                    "output_data": ["todo_content", "todo_file_path", "file_exists"],
                },
                "GENERATE": {
                    "module_path": "gai.asm.states",
                    "class_name": "PureActionState",
                    "title": "GENERATE",
                    "action": "generate_update_instructions",
                    "output_data": ["update_instruction"],
                },
                "UPDATE_TODO": {
                    "module_path": "gai.asm.states",
                    "class_name": "UpdateToDoState",
                    "title": "UPDATE_TODO",
                    "input_data": {
                        "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                        "todo_data": {"type": "state_bag", "dependency": "todo_content"},
                        "update_instruction": {"type": "state_bag", "dependency": "update_instruction"},
                        "todo_path": {"type": "state_bag", "dependency": "todo_path"},
                        "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
                    },
                    "output_data": ["streamer", "get_assistant_message", "updated_todo", "todo_file_path"],
                },
                "FINAL": {
                    "output_data": ["monologue", "updated_todo", "todo_content", "update_instruction"],
                },
            },
            get_todo_path=lambda state: non_existent_path,
            get_llm_config=lambda state: {
                "client_type": "anthropic",
                "model": "claude-sonnet-4-20250514",
                "max_tokens": 32000,
                "temperature": 0.7,
                "top_p": 0.95,
                "tools": True,
            },
            get_mcp_client=lambda state: McpAggregatedClient([]),
            generate_update_instructions=generate_update_instructions,
            monologue=monologue,
        )
    
    return fsm

# Run the error handling workflow
try:
    error_fsm = await run_error_handling_workflow()
    
    # Step 1: READ_TODO (should handle non-existent file gracefully)
    print("📖 Step 1: READ_TODO with non-existent file...")
    await error_fsm.run_async()
    
    file_exists = error_fsm.state_bag.get("file_exists")
    todo_content = error_fsm.state_bag.get("todo_content")
    
    print(f"   File exists: {file_exists}")
    print(f"   Content: {todo_content}")
    
    if not file_exists and todo_content is None:
        print("   ✅ Correctly handled non-existent file")
    else:
        print("   ❌ Unexpected result for non-existent file")
    
    # Step 2: GENERATE (should work with empty content)
    print("\n🎯 Step 2: GENERATE with empty content...")
    error_fsm.restart()  # Reset state machine history
    await error_fsm.run_async()
    
    update_instruction = error_fsm.state_bag.get("update_instruction")
    print(f"   Generated instruction: {update_instruction}")
    
    if update_instruction:
        print("   ✅ Successfully generated instruction for empty content")
    else:
        print("   ❌ Failed to generate instruction")
    
    # Step 3: UPDATE_TODO (should create new content)
    print("\n✏️  Step 3: UPDATE_TODO creating new content...")
    error_fsm.restart()  # Reset state machine history
    await error_fsm.run_async()
    
    # Don't stream for brevity in error test
    print("   📝 UPDATE_TODO completed")
    
    # Step 4: FINAL
    print("\n🏁 Step 4: FINAL...")
    error_fsm.restart()  # Reset state machine history
    await error_fsm.run_async()
    
    # Check if file was created
    final_file_path = error_fsm.state_bag.get("todo_file_path")
    file_created = os.path.exists(final_file_path) if final_file_path else False
    
    print(f"   Final file path: {final_file_path}")
    print(f"   File created: {file_created}")
    
    if file_created:
        print("   ✅ Successfully created new todo file from scratch")
    else:
        print("   ❌ Failed to create new todo file")
    
    print("\n🎉 Error handling workflow completed successfully!")
    
except Exception as e:
    print(f"❌ Error in workflow: {e}")

---

## Test Case 3: Custom Update Instructions

Test the workflow with predefined update instructions instead of LLM-generated ones.


In [ ]:
print("=== Test Case 3: Custom Update Instructions Workflow ===")

# Create a different todo file for this test
custom_todo_content = """# Weekly Sprint Goals

## Development Tasks
- [ ] Implement OAuth2 authentication
- [ ] Create REST API endpoints for user management
- [ ] Add database migrations for new schema

## Testing Tasks  
- [ ] Write unit tests for authentication module
- [ ] Perform integration testing
- [ ] Load testing for API endpoints

## Documentation
- [ ] Update API documentation
- [ ] Create user guide for new features
- [x] Technical architecture review completed

## Deployment
- [ ] Set up staging environment
- [ ] Configure CI/CD pipeline
- [ ] Production deployment checklist
"""

# Create temporary file for custom test
with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False) as f:
    f.write(custom_todo_content)
    custom_todo_path = f.name

print(f"Created custom test todo file at: {custom_todo_path}")

# Define custom generate function that uses predefined instructions
async def generate_custom_instructions(state):
    """Use predefined custom update instructions."""
    custom_instruction = """
    Please update this sprint todo list with the following changes:
    1. Mark the OAuth2 authentication task as completed
    2. Mark the unit tests for authentication as completed  
    3. Add a new high-priority task: "Fix security vulnerability in user session handling"
    4. Add a new documentation task: "Create API security best practices guide"
    5. Update the technical architecture review to include completion date
    6. Add progress indicators showing 40% completion for the sprint
    """
    
    state.machine.state_bag["update_instruction"] = custom_instruction
    print(f"Using custom instruction: {custom_instruction}")

async def run_custom_workflow():
    """Run workflow with custom update instructions."""
    
    monologue = Monologue()
    
    with AgenticStateMachine.StateMachineBuilder(
        """
        INIT --> READ_TODO
        READ_TODO --> GENERATE
        GENERATE --> UPDATE_TODO
        UPDATE_TODO --> FINAL
        """
    ) as builder:
        fsm = builder.build(
            {
                "INIT": {
                    "input_data": {
                        "todo_path": {"type": "getter", "dependency": "get_todo_path"},
                        "llm_config": {"type": "getter", "dependency": "get_llm_config"},
                        "mcp_client": {"type": "getter", "dependency": "get_mcp_client"},
                    }
                },
                "READ_TODO": {
                    "module_path": "gai.asm.states",
                    "class_name": "ReadToDoState",
                    "title": "READ_TODO",
                    "input_data": {
                        "todo_path": {"type": "state_bag", "dependency": "todo_path"},
                    },
                    "output_data": ["todo_content", "todo_file_path", "file_exists"],
                },
                "GENERATE": {
                    "module_path": "gai.asm.states",
                    "class_name": "PureActionState",
                    "title": "GENERATE",
                    "action": "generate_custom_instructions",
                    "output_data": ["update_instruction"],
                },
                "UPDATE_TODO": {
                    "module_path": "gai.asm.states",
                    "class_name": "UpdateToDoState",
                    "title": "UPDATE_TODO",
                    "input_data": {
                        "llm_config": {"type": "state_bag", "dependency": "llm_config"},
                        "todo_data": {"type": "state_bag", "dependency": "todo_content"},
                        "update_instruction": {"type": "state_bag", "dependency": "update_instruction"},
                        "todo_path": {"type": "state_bag", "dependency": "todo_path"},
                        "mcp_client": {"type": "state_bag", "dependency": "mcp_client"},
                    },
                    "output_data": ["streamer", "get_assistant_message", "updated_todo", "todo_file_path"],
                },
                "FINAL": {
                    "output_data": ["monologue", "updated_todo", "todo_content", "update_instruction"],
                },
            },
            get_todo_path=lambda state: custom_todo_path,
            get_llm_config=lambda state: {
                "client_type": "anthropic",
                "model": "claude-sonnet-4-20250514",
                "max_tokens": 32000,
                "temperature": 0.3,  # Lower temperature for more consistent updates
                "top_p": 0.95,
                "tools": True,
            },
            get_mcp_client=lambda state: McpAggregatedClient([]),
            generate_custom_instructions=generate_custom_instructions,
            monologue=monologue,
        )
    
    return fsm

try:
    # Run the custom workflow
    custom_fsm = await run_custom_workflow()
    
    # Execute all states in sequence
    print("📖 Executing READ_TODO...")
    await custom_fsm.run_async()
    
    print("🎯 Executing GENERATE with custom instructions...")
    await custom_fsm.run_async()
    
    print("✏️  Executing UPDATE_TODO...")
    await custom_fsm.run_async()
    
    print("🏁 Executing FINAL...")
    await custom_fsm.run_async()
    
    # Show results
    initial_content = custom_fsm.state_bag.get("todo_content")
    updated_todo = custom_fsm.state_bag.get("updated_todo")
    final_content = updated_todo.get("updated_content", "") if updated_todo else ""
    
    print("\n" + "="*60)
    print("CUSTOM WORKFLOW RESULTS")
    print("="*60)
    
    print("📋 Initial Content (first 200 chars):")
    print(initial_content[:200] + "..." if len(initial_content) > 200 else initial_content)
    
    print(f"\n📏 Content Length Comparison:")
    print(f"   Initial: {len(initial_content)} characters")
    print(f"   Updated: {len(final_content)} characters")
    print(f"   Change: {len(final_content) - len(initial_content):+d} characters")
    
    if final_content:
        print(f"\n📋 Updated Content:")
        print("-" * 50)
        print(final_content)
        print("-" * 50)
    
    print("\n✅ Custom workflow completed successfully!")
    
finally:
    # Clean up custom test file
    try:
        os.unlink(custom_todo_path)
        print(f"\n🧹 Cleaned up custom test file: {custom_todo_path}")
    except:
        pass

In [ ]:
# Final cleanup of the main test file
try:
    os.unlink(workflow_todo_path)
    print(f"🧹 Cleaned up main test file: {workflow_todo_path}")
except:
    pass

print("\n🎉 All Update Read Todo State workflow tests completed successfully!")
print("\n📊 Summary of Tests:")
print("   ✅ Test 1: Complete workflow with existing file")
print("   ✅ Test 2: Error handling with non-existent file") 
print("   ✅ Test 3: Custom update instructions workflow")
print("\n🔄 State Flow Demonstrated:")
print("   INIT → READ_TODO → GENERATE → UPDATE_TODO → FINAL")